In [317]:
"""
Новая версия.
"""

'\nНовая версия.\n'

In [318]:
import tkinter
import sys
import pandas as pd
import openpyxl
import numpy as np
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font
from openpyxl.styles import Alignment
import os
from tkinter import *
from tkinter import filedialog
from tkinter import messagebox
from tkinter import ttk
import time
import datetime
# pd.options.mode.chained_assignment = None  # default='warn'
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
pd.options.mode.chained_assignment = None  # default='warn'


In [319]:
name_file_abiturs = 'data/abitur.xlsx'
name_file_person = 'data/person.xlsx'
path_to_end_folder_report = 'data'

In [320]:
#создаем датафрейм со специальностями
lst_code = ['23.01.09','43.01.06','23.02.06','43.02.06','15.01.05','15.01.35','15.01.33','23.01.10'
    ,'08.01.31','23.01.17','08.02.09','23.02.07','13.02.07','35.01.27']

lst_name_spec = ['Машинист локомотива','Проводник на железнодорожном транспорте','Техническая эксплуатация подвижного состава железных дорог',
                 'Сервис на транспорте (по видам транспорта)','Сварщик (ручной и частично механизированной сварки (наплавки)',
                 'Мастер слесарных работ','Токарь на станках с числовым программным управлением','Слесарь по обслуживанию и ремонту подвижного состава'
    ,'Электромонтажник электрических сетей и электрооборудования','Мастер по ремонту и обслуживанию автомобилей',
                 'Монтаж, наладка и эксплуатация электрооборудования промышленных и гражданских зданий',
                 'Техническое обслуживание и ремонт двигателей, систем и агрегатов автомобилей','Электроснабжение (по отраслям)',
                 'Мастер сельскохозяйственного производства']

lst_plan = [100,25,25,25,50,25,25,50
    ,25,25,25,25,25,25]
base_df = pd.DataFrame(columns=['Код','Наименование'])
base_df['Код'] = lst_code
base_df['Наименование'] = lst_name_spec
base_df['Направление подготовки'] = base_df['Код'] + ' ' + base_df['Наименование']
base_df['База'] = '9 кл.'
base_df['Количество мест'] = lst_plan

In [321]:
df_abitur = pd.read_excel(name_file_abiturs, skiprows=3, usecols=['Абитуриент', 'Доп. статус','№ заявления'])
df_person = pd.read_excel(name_file_person, sheet_name='Абитуриенты', skiprows=8,
                          usecols=['ФИО', 'Нуждается в общежитии', 'Формирующее подр.',
                                   'Направление подготовки', 'Сдан оригинал','Состояние выбран. конкурса','СНИЛС'])



In [322]:
df_person = df_person[~df_person['Направление подготовки'].isnull()]  # убираем тех у кого нет заявлений
df_abitur = df_abitur[~df_abitur['№ заявления'].isnull()]



In [323]:
ts =df_person[df_person['Направление подготовки'] =='23.01.09 Машинист локомотива']
ts.shape

(169, 7)

In [324]:
df_dupl = df_person.drop_duplicates(subset=['ФИО'])  # создаем датафрейм без дубликатов

In [325]:



dupl_cross_df = df_dupl.merge(df_abitur, how='inner', left_on='ФИО', right_on='Абитуриент')


In [326]:
# Преобразовываем да-нет в 1 или 0 для подсчетов
dupl_cross_df['Нуждается в общежитии'] = dupl_cross_df['Нуждается в общежитии'].apply(
    lambda x: 0 if x == 'нет' else 1)
dupl_cross_df['Сдан оригинал'] = dupl_cross_df['Сдан оригинал'].apply(lambda x: 0 if x == 'нет' else 1)
# заменяем нан на пустые строки чтобы произвести поиск слова сирота;
dupl_cross_df['Доп. статус'].fillna('', inplace=True)
dupl_cross_df['Сироты'] = dupl_cross_df['Доп. статус'].apply(lambda x: 1 if 'Сирота;' in x else 0)
dupl_cross_df['СВО'] = dupl_cross_df['Доп. статус'].apply(
    lambda x: 1 if 'Дети военнослужащих, участвующих в спецоперации' in x else 0)
dupl_cross_df['Целевой договор'] = dupl_cross_df['Доп. статус'].apply(lambda x: 1 if 'Целевой договор' in x else 0)

dupl_cross_df['for_counting'] = 1

dupl_cross_df.drop(columns=['Доп. статус'], inplace=True)

In [327]:
dupl_svod_df = pd.DataFrame.pivot_table(dupl_cross_df,
                                        index=['Формирующее подр.', 'Направление подготовки'],
                                        values=['Сдан оригинал', 'Сироты', 'СВО','Целевой договор',
                                                'Нуждается в общежитии'],
                                        aggfunc='sum')

dupl_svod_df.columns = ['Нуждается в общежитии чел.', 'Дети СВО', 'Сдано оригиналов', 'Сирот чел.','Целевой договор']

In [328]:
# Меняем местами столбцы
single_out_df = dupl_svod_df.reindex(
    columns=['Сдано оригиналов',
             'Нуждается в общежитии чел.',
             'Сирот чел.', 'Дети СВО','Целевой договор'])

In [329]:
# Соединяем оба датафрейма

cross_df = df_person.merge(df_abitur, how='inner', left_on='ФИО', right_on='Абитуриент')

In [330]:
ts[ts.duplicated(['ФИО'], keep=False)]

,ФИО,СНИЛС,Нуждается в общежитии,Формирующее подр.,Направление подготовки,Состояние выбран. конкурса,Сдан оригинал
600,Ринчинов Зорикто Базарович,148-473-709-97,нет,Отделение железнодорожного транспорта,23.01.09 Машинист локомотива,Сданы ВИ,да
601,Ринчинов Зорикто Базарович,148-473-709-97,нет,Отделение железнодорожного транспорта,23.01.09 Машинист локомотива,Забрал документы,да


In [331]:
# Преобразовываем да-нет в 1 или 0 для подсчетов
cross_df['Нуждается в общежитии'] = cross_df['Нуждается в общежитии'].apply(lambda x: 0 if x == 'нет' else 1)
cross_df['Сдан оригинал'] = cross_df['Сдан оригинал'].apply(lambda x: 0 if x == 'нет' else 1)

In [332]:
# заменяем нан на пустые строки чтобы произвести поиск слова сирота;
cross_df['Доп. статус'].fillna('', inplace=True)
cross_df['Сироты'] = cross_df['Доп. статус'].apply(lambda x: 1 if 'Сирота;' in x else 0)
cross_df['СВО'] = cross_df['Доп. статус'].apply(
    lambda x: 1 if 'Дети военнослужащих, участвующих в спецоперации' in x else 0)

In [333]:
cross_df['for_counting'] = 1

cross_df.drop(columns=['Доп. статус'], inplace=True)

In [334]:
# Создаем сокращенный датафрейм чтобы добавить его в базовый
small_df = cross_df[['Направление подготовки','Состояние выбран. конкурса','for_counting']]

In [335]:
# объединяем датафреймы
union_df = base_df.merge(small_df,how='outer',left_on='Направление подготовки',right_on='Направление подготовки')
union_df.fillna(0,inplace=True)

In [336]:
# забранные заявления
return_z = union_df[union_df['Состояние выбран. конкурса'] =='Забрал документы' ]

In [337]:
base_df_groupby = union_df.groupby(['Направление подготовки']).agg({'for_counting':sum})
base_df_groupby['for_counting'] = base_df_groupby['for_counting'].apply(int)
base_df_groupby = base_df_groupby.reset_index()
base_df_groupby.rename(columns={'for_counting':'Подано заявлений'},inplace=True)

In [338]:
base_df = base_df.merge(base_df_groupby,how='inner',left_on='Направление подготовки',right_on='Направление подготовки')
base_df.sort_values(by='Подано заявлений',ascending=False,inplace=True)

In [339]:
base_df

,Код,Наименование,Направление подготовки,База,Количество мест,Подано заявлений
0,23.01.09,Машинист локомотива,23.01.09 Машинист локомотива,9 кл.,100,170
4,15.01.05,Сварщик (ручной и частично механизированной св...,15.01.05 Сварщик (ручной и частично механизиро...,9 кл.,50,95
1,43.01.06,Проводник на железнодорожном транспорте,43.01.06 Проводник на железнодорожном транспорте,9 кл.,25,77
9,23.01.17,Мастер по ремонту и обслуживанию автомобилей,23.01.17 Мастер по ремонту и обслуживанию авто...,9 кл.,25,74
3,43.02.06,Сервис на транспорте (по видам транспорта),43.02.06 Сервис на транспорте (по видам трансп...,9 кл.,25,69
7,23.01.10,Слесарь по обслуживанию и ремонту подвижного с...,23.01.10 Слесарь по обслуживанию и ремонту под...,9 кл.,50,65
6,15.01.33,Токарь на станках с числовым программным управ...,15.01.33 Токарь на станках с числовым программ...,9 кл.,25,58
12,13.02.07,Электроснабжение (по отраслям),13.02.07 Электроснабжение (по отраслям),9 кл.,25,39
8,08.01.31,Электромонтажник электрических сетей и электро...,08.01.31 Электромонтажник электрических сетей ...,9 кл.,25,34
10,08.02.09,"Монтаж, наладка и эксплуатация электрооборудов...","08.02.09 Монтаж, наладка и эксплуатация электр...",9 кл.,25,33


In [340]:
base_df.rename(columns={'Наименование':'Наименование образовательной программы'})
base_df.drop(columns='Направление подготовки',inplace=True)

In [341]:
# считаем количество тех кто забрал документы
cross_df['Забрали заявления'] = cross_df['Состояние выбран. конкурса'].apply(lambda x: 1 if x=='Забрал документы' else 0)
cross_df['Заявления'] = cross_df['for_counting']

In [342]:
svod_df = pd.DataFrame.pivot_table(cross_df,
                                           index=['Формирующее подр.', 'Направление подготовки'],
                                           values=['Заявления', 'Забрали заявления',],
                                           aggfunc='sum')

In [343]:
svod_df=svod_df.reindex(columns=['Заявления','Забрали заявления'])

In [344]:
svod_df['Итого заявлений'] = svod_df['Заявления'] - svod_df['Забрали заявления']

In [345]:
out_df = svod_df.reset_index()

In [346]:
single_out_df = single_out_df.reset_index()

In [347]:
finish_df = pd.merge(out_df, single_out_df, how='outer')  # объединяем

In [348]:
wb = openpyxl.Workbook()
# Переименовываем лист
sheet = wb['Sheet']
sheet.title = 'Отчет'

sum_row = finish_df.sum(axis=0).to_frame().transpose()

sum_row['Формирующее подр.'] = 'Всего'
sum_row['Направление подготовки'] = ''

# объединяем датафреймы

all_finish_df = pd.concat([finish_df, sum_row], axis=0)

for r in dataframe_to_rows(all_finish_df, index=False, header=True):
    if len(r) != 1:
        wb['Отчет'].append(r)

# # Настраиваем выходной файл
wb['Отчет'].column_dimensions['A'].width = 30
wb['Отчет'].column_dimensions['B'].width = 50
wb['Отчет']['B2'].alignment = Alignment(wrap_text=True)
wb['Отчет'].column_dimensions['C'].width = 20
wb['Отчет'].column_dimensions['D'].width = 20
wb['Отчет'].column_dimensions['F'].width = 20
wb['Отчет'].column_dimensions['G'].width = 20
wb['Отчет'].column_dimensions['H'].width = 30
wb['Отчет']['H1'].alignment = Alignment(wrap_text=True)

# Получаем текущее время для того чтобы использовать в названии
t = time.localtime()
current_time = time.strftime('%H_%M_%d_%m', t)
# Сохраняем итоговый файл
base_df.to_excel(f'{path_to_end_folder_report}/Количество поданых заявлений {current_time}.xlsx',index=False)
wb.save(f'{path_to_end_folder_report}/Ежедневный отчет приемной комиссии ГБПОУ БРИТ {current_time}.xlsx')

In [349]:
# ищем полных тезок
temp_dupl_df = df_person.drop_duplicates(subset=['ФИО','СНИЛС'])

In [351]:
tezki_df = temp_dupl_df[temp_dupl_df.duplicated(subset='ФИО',keep=False)]

In [352]:
tezki_df.to_excel(f'{path_to_end_folder_report}/Полные тезки {current_time}.xlsx',index=False)